# Guía de inicio rápido

Este tutorial le brindará una visión general y concisa sobre cómo construir una aplicación de modelo de lenguaje de extremo a extremo utilizando LangChain.

>Docs: https://python.langchain.com/en/latest/getting_started/getting_started.html

**LangChain** es una biblioteca Python, creada por **Harrison Chase**, de código abierto diseñada específicamente para proporcionar a los desarrolladores las herramientas necesarias para crear aplicaciones basadas en grandes modelos lingüísticos (LLM) de manera eficiente y profesional.

Con LangChain, los desarrolladores pueden conectarse fácilmente a diversas fuentes de datos y de cálculo, permitiéndoles crear aplicaciones que realicen tareas de procesamiento del lenguaje natural (PLN) en fuentes de datos específicas de dominio, repositorios privados y mucho más.

Es un marco para la creación de agentes capaces de razonar y encadenar tareas. Brindando a los desarrolladores la capacidad de crear agentes inteligentes capaces de razonar sobre problemas complejos y dividirlos en subtareas más pequeñas. Con LangChain, es posible introducir contexto y memoria en las finalizaciones al crear pasos intermedios y encadenar comandos.

## Instalación
Para empezar, instala LangChain con el siguiente comando:

In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


## Configuración del entorno
Usar LangChain normalmente requerirá integraciones con uno o más proveedores de modelos, almacenes de datos, apis, etc.
Para este ejemplo, vamos a utilizar las APIs de **OpenAI**, por lo que primero tendremos que instalar su SDK:

In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


A continuación, tendremos que establecer la variable de entorno en nuestro archivo .env

In [4]:
pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

## Creación de una aplicación de modelos lingüísticos
Ahora que hemos instalado LangChain y configurado nuestro entorno, podemos empezar a construir nuestra aplicación de modelo de lenguaje.

LangChain proporciona muchos módulos que pueden ser utilizados para construir aplicaciones de modelos de lenguaje. Los módulos pueden combinarse para crear aplicaciones más complejas, o utilizarse individualmente para aplicaciones sencillas.

### LLMs: Obtener predicciones de un modelo lingüístico
El bloque de construcción más básico de LangChain es llamar a un LLM con algún input. Básicamente tener tu propio ChatGPT corriendo en tu terminal. 
>Consejo: Compara los precios del servicio ChatGPT Plus de OpenAI y sus precios para la API 😜

En esta lección aprenderemos a interactuar con ChatGPT para generar texto. Muchas aplicaciones que están valoradas en millones de dólares como JasperAI, CopyAI... hacen lo mismo como parte de su negocio principal.

Veamos un ejemplo sencillo de cómo hacerlo. Para ello, primero tenemos que importar el LLM wrapper.

In [6]:
from langchain.llms import OpenAI

Podemos entonces inicializar el wrapper con cualquier argumento. En este ejemplo, probablemente queremos que las salidas sean standard, así que lo inicializaremos con una temperatura media/baja

In [7]:
llm = OpenAI(temperature=0.3)

¡Ahora podemos llamarlo con algún input!

In [8]:
text = "What would be a good company name for a company that makes Autonomous Agents?"
print(llm(text))



Autonomous Agents Solutions Inc.


### Plantillas de Promps: Gestión de prompts para LLM

Llamar a un LLM es un gran primer paso, pero es sólo el principio. Normalmente, cuando utilizas un LLM en una aplicación, no envías el input del usuario directamente al LLM. En su lugar, es probable que tomes el input del usuario, construyas un prompt y luego lo envíes al LLM.

Por ejemplo, en el ejemplo anterior, el texto que pasamos estaba "hardcoded" para pedir el nombre de una empresa que fabrica agentes autónomos. En este servicio, lo que querríamos hacer es tomar sólo la entrada del usuario describiendo lo que hace la empresa, y luego formatear el prompt con esa información.

Esto es fácil de hacer con LangChain.

Primero definamos la plantilla del prompt:

In [6]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [7]:
print(prompt.format(product="autonomous agent"))

What is a good name for a company that makes autonomous agent?


In [9]:
print(llm(prompt.format(product="autonomous agent")))



Autonomous Agents, Inc.


### Cadenas: Combina LLMs y prompts en flujos de trabajo de varios pasos.

Hasta ahora, hemos trabajado con los PromptTemplate y LLM de base por sí solos. Pero, por supuesto, una aplicación real no es sólo una primitiva, sino más bien una combinación de ellas.

Una cadena en LangChain se compone de enlaces, que pueden ser primitivas como LLMs u otras cadenas.

El tipo más básico de cadena es una LLMChain, que consiste en un PromptTemplate y un LLM.

Extendiendo el ejemplo anterior, podemos construir una LLMChain que tome la entrada del usuario, la formatee con un PromptTemplate, y luego pase la respuesta formateada a un LLM.

In [10]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [11]:
llm = OpenAI(temperature=0.3)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

Ahora podemos crear una cadena muy simple que tomará la entrada del usuario, formateará el prompt con ella, y luego la enviará al LLM:

In [12]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
print(chain.run("Software as a service"))



CloudSoft Solutions.


Ya está. Esta es la primera cadena: una LLM Chain. Este es uno de los tipos más simples de cadenas, pero entender cómo funciona te preparará para trabajar con cadenas más complejas.

### Agentes: Cadenas de llamadas dinámicas basadas en el input del usuario

Hasta ahora, las cadenas que hemos visto se ejecutaban en un orden predeterminado.

Los agentes ya no lo hacen: utilizan un LLM para determinar qué acciones realizar y en qué orden. Una acción puede consistir en utilizar una herramienta y observar su resultado, o en volver al usuario.

Cuando se utilizan correctamente, los agentes pueden ser extremadamente potentes. En este tutorial, le mostramos cómo utilizar fácilmente los agentes a través de la API más simple y de más alto nivel.

Para cargar agentes, debe comprender los siguientes conceptos:

- **Herramienta**: Una función que realiza una tarea específica. Pueden ser cosas como: Búsqueda en Google, búsqueda en bases de datos, Python REPL, otras cadenas. La interfaz para una herramienta es actualmente una función que se espera que tenga una cadena como entrada, con una cadena como salida.

- **LLM**: El modelo de lenguaje que alimenta al agente.

- **Agente**: El agente a utilizar. Debe ser una cadena que haga referencia a una clase de agente de soporte. Dado que este cuaderno se centra en la API más simple y de más alto nivel, sólo cubre el uso de los agentes estándar soportados. Si desea implementar un agente personalizado, consulte la documentación https://python.langchain.com/en/latest/getting_started/getting_started.html#agents-dynamically-call-chains-based-on-user-input

Para este ejemplo, también necesitarás instalar el paquete SerpAPI Python.

In [ ]:
pip install google-search-results

In [ ]:
# https://serpapi.com/
import os
os.environ["SERPAPI_API_KEY"] = "..."

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
# En primer lugar, vamos a cargar el modelo lingüístico que vamos a utilizar para controlar el agente.
llm = OpenAI(temperature=0)

# A continuación, vamos a cargar algunas herramientas a utilizar. Ten en cuenta que la herramienta `llm-math` utiliza un LLM, así que tenemos que pasarlo.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Por último, vamos a inicializar un agente con las herramientas, el modelo de lenguaje y el tipo de agente que queremos utilizar.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# ¡Ahora vamos a probarlo!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

### Memoria: Añadir estado a cadenas y agentes

Hasta ahora, todas las cadenas y agentes por los que hemos pasado han sido sin estado. Pero a menudo, es posible que quieras que una cadena o agente tenga algún concepto de "memoria" para que pueda recordar información sobre sus interacciones anteriores. El ejemplo más claro y sencillo de esto es el diseño de un chatbot: quieres que recuerde mensajes anteriores para que pueda utilizar su contexto para mantener una conversación mejor. Esto sería un tipo de "memoria a corto plazo". En el lado más complejo, se podría imaginar una cadena/agente que recordara piezas clave de información a lo largo del tiempo: esto sería una forma de "memoria a largo plazo".

In [ ]:
from langchain import OpenAI, ConversationChain

In [ ]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

In [ ]:
output = conversation.predict(input="Hi there!")
print(output)

## Creación de una aplicación de modelos lingüísticos: Modelos de chat

Del mismo modo, puedes utilizar modelos de chat en lugar de LLM. Los modelos de chat son una variación de los modelos de lenguaje. Aunque los modelos de chat utilizan modelos lingüísticos, la interfaz que exponen es un poco diferente: en lugar de exponer una API de "entrada de texto, salida de texto", exponen una interfaz en la que los "mensajes de chat" son las entradas y salidas.

Las API de modelos de chat son bastante nuevas.

### Plantillas de mensajes de chat

De forma similar a los LLMs, puedes hacer uso de plantillas utilizando un MessagePromptTemplate.

Puedes construir un ChatPromptTemplate a partir de uno o más MessagePromptTemplates. Puede utilizar el format_prompt de ChatPromptTemplate - esto devuelve un PromptValue, que puede convertir en una cadena o en un objeto Message, dependiendo de si desea utilizar el valor formateado como entrada a un modelo llm o chat.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
# Obtén una finalización de chat a partir de los mensajes formateados
chat(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

### Cadenas con modelos de chat
La LLMChain comentada en la sección anterior también puede utilizarse con modelos de chat:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="French", text="I love programming.")

### Agentes con modelos de chat
Los agentes también se pueden utilizar con modelos de chat, puedes inicializar uno utilizando AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION como tipo de agente.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [ ]:
# En primer lugar, vamos a cargar el modelo de lenguaje que vamos a utilizar para controlar el agente.
chat = ChatOpenAI(temperature=0)

# A continuación, vamos a cargar algunas herramientas a utilizar. Ten en cuenta que la herramienta `llm-math` utiliza un LLM, así que tenemos que pasarlo.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Por último, vamos a inicializar un agente con las herramientas, el modelo de lenguaje y el tipo de agente que queremos utilizar.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# ¡Ahora vamos a probar!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")

### Memoria: Añadir estado a cadenas y agentes
Puedes utilizar la memoria con cadenas y agentes inicializados con modelos de chat. La principal diferencia entre esto y y l amemoria para LLMs es que en lugar de intentar condensar todos los mensajes anteriores en una cadena, podemos mantenerlos como su propio objeto de memoria único.

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

In [ ]:
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

In [ ]:
conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

In [ ]:
conversation.predict(input="Tell me about yourself.")